## Worldometer Site Web Scrapping

---

In [1]:
# Import libraries

import os
import re
import glob
import requests 
import numpy as np
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup

In [2]:
# Import dataset

link = 'https://www.worldometers.info/coronavirus/'
req = requests.get(link)
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
# Find table

thead = soup.find_all('thead')[-1]
head = thead.find_all('tr')
tbody = soup.find_all('tbody')[0]
body = tbody.find_all('tr')

In [4]:
# Get the table contents

head_rows = []
body_rows = []

for tr in head:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    head_rows.append(row)
# print(head_rows)

for tr in body:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    body_rows.append(row)
print(head_rows)

[['#', 'Country,Other', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths', 'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical', 'Tot\xa0Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/\n1M pop\n', 'Population', 'Continent', '1 Caseevery X ppl', '1 Deathevery X ppl', '1 Testevery X ppl']]


In [5]:
# Save content in dataframe

df_wm = pd.DataFrame(body_rows[:len(body_rows)-6], 
                     columns=head_rows[0])         

df_wm.tail(5)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
215,208,Greenland,17,,,,17,,0,,299,,"11,875","209,045","56,806",North America,"3,342",,5
216,209,Saint Pierre Miquelon,16,,,,12,,4,,"2,766",,"2,624","453,665","5,784",North America,362,,2
217,210,Solomon Islands,16,,,,5,,11,,23,,115,166,"692,856",Australia/Oceania,"43,304",,"6,025"
218,211,Montserrat,13,,1,,12,,0,,"2,604",200,577,"115,562","4,993",North America,384,"4,993",9
219,212,Falkland Islands,13,,,,13,,0,,"3,697",,"3,417","971,843","3,516",South America,270,,1


In [6]:
# Drop unwanted rows & columns

df_wm = df_wm.iloc[8:, :-3].reset_index(drop=True)
df_wm = df_wm.drop('#', axis=1)
df_wm.head()

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent
0,USA,"10,557,037","+133,427","245,777","+1,322","6,595,348","+41,959","3,715,912","19,077","31,826",741,"160,475,931","483,789","331,706,290",North America
1,India,"8,635,754","+44,679","127,615",+511,"8,011,844","+54,638","496,295","8,944","6,236",92,"119,615,857","86,371","1,384,900,866",Asia
2,Brazil,"5,701,283","+25,517","162,842",+204,"5,064,344",,"474,097","8,318","26,753",764,"21,900,000","102,766","213,106,404",South America
3,France,"1,829,659","+22,180","42,207",+857,"131,920","+2,185","1,655,532","4,750","28,008",646,"18,249,032","279,353","65,326,081",Europe
4,Russia,"1,817,109","+20,977","31,161",+368,"1,350,741","+15,600","435,207","2,300","12,450",213,"65,600,000","449,447","145,957,281",Europe


In [7]:
# Rename & rearrange columns
df_wm.columns = ['Country/Region', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop',
       'Population', 'Continent']

df_wm = df_wm[['Country/Region', 'Continent', 'Population', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop' ]]

df_wm.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop
0,USA,North America,"331,706,290","10,557,037","+133,427","245,777","+1,322","6,595,348","+41,959","3,715,912","19,077","31,826",741,"160,475,931","483,789"
1,India,Asia,"1,384,900,866","8,635,754","+44,679","127,615",+511,"8,011,844","+54,638","496,295","8,944","6,236",92,"119,615,857","86,371"
2,Brazil,South America,"213,106,404","5,701,283","+25,517","162,842",+204,"5,064,344",,"474,097","8,318","26,753",764,"21,900,000","102,766"
3,France,Europe,"65,326,081","1,829,659","+22,180","42,207",+857,"131,920","+2,185","1,655,532","4,750","28,008",646,"18,249,032","279,353"
4,Russia,Europe,"145,957,281","1,817,109","+20,977","31,161",+368,"1,350,741","+15,600","435,207","2,300","12,450",213,"65,600,000","449,447"


In [8]:
who_region = {}

# African Region AFRO
afro = "Algeria, Angola, Cabo Verde, Congo, DRC, Eswatini, Sao Tome and Principe, Benin, South Sudan, Western Sahara, Congo (Brazzaville), Congo (Kinshasa), Cote d'Ivoire, Botswana, Burkina Faso, Burundi, Cameroon, Cape Verde, Central African Republic, Chad, Comoros, Ivory Coast, Democratic Republic of the Congo, Equatorial Guinea, Eritrea, Ethiopia, Gabon, Gambia, Ghana, Guinea, Guinea-Bissau, Kenya, Lesotho, Liberia, Madagascar, Malawi, Mali, Mauritania, Mauritius, Mozambique, Namibia, Niger, Nigeria, Republic of the Congo, Rwanda, São Tomé and Príncipe, Senegal, Seychelles, Sierra Leone, Somalia, South Africa, Swaziland, Togo, Uganda, Tanzania, Zambia, Zimbabwe"
afro = [i.strip() for i in afro.split(',')]
for i in afro:
    who_region[i] = 'Africa'
    
# Region of the Americas PAHO
paho = 'Antigua and Barbuda, Argentina, Bahamas, Barbados, Belize, Bermuda, Bolivia, Brazil, Canada, Chile, Colombia, Costa Rica, Cuba, Dominica, Dominican Republic, Ecuador, El Salvador, Grenada, Guatemala, Guyana, Haiti, Honduras, Jamaica, Mexico, Nicaragua, Panama, Paraguay, Peru, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines, Suriname, Trinidad and Tobago, United States, US, USA, Uruguay, Venezuela'
paho = [i.strip() for i in paho.split(',')]
for i in paho:
    who_region[i] = 'Americas'

# South-East Asia Region SEARO
searo = 'Bangladesh, Bhutan, North Korea, India, Indonesia, Maldives, Myanmar, Burma, Nepal, Sri Lanka, Thailand, Timor-Leste'
searo = [i.strip() for i in searo.split(',')]
for i in searo:
    who_region[i] = 'South-East Asia'

# European Region EURO
euro = 'Albania, Andorra, Greenland, Kosovo, Holy See, Vatican City, Liechtenstein, Armenia, Czechia, Austria, Azerbaijan, Belarus, Belgium, Bosnia and Herzegovina, Bulgaria, Croatia, Cyprus, Czech Republic, Denmark, Estonia, Finland, France, Georgia, Germany, Greece, Hungary, Iceland, Ireland, Israel, Italy, Kazakhstan, Kyrgyzstan, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Netherlands, North Macedonia, Norway, Poland, Portugal, Moldova, Romania, Russia, San Marino, Serbia, Slovakia, Slovenia, Spain, Sweden, Switzerland, Tajikistan, Turkey, Turkmenistan, Ukraine, United Kingdom, UK, Uzbekistan'
euro = [i.strip() for i in euro.split(',')]
for i in euro:
    who_region[i] = 'Europe'

# Eastern Mediterranean Region EMRO
emro = 'Afghanistan, Bahrain, Djibouti, Egypt, Iran, Iraq, Jordan, Kuwait, Lebanon, Libya, Morocco, Oman, Pakistan, Palestine, West Bank and Gaza, Qatar, Saudi Arabia, Somalia, Sudan, Syria, Tunisia, United Arab Emirates, UAE, Yemen'
emro = [i.strip() for i in emro.split(',')]
for i in emro:
    who_region[i] = 'Eastern Mediterranean'

# Western Pacific Region WPRO
wpro = 'Australia, Brunei, Cambodia, China, Cook Islands, Fiji, Japan, Hong Kong, Kiribati, Laos, Malaysia, Marshall Islands, Micronesia, Mongolia, Nauru, New Zealand, Niue, Palau, Papua New Guinea, Philippines, South Korea, S. Korea, Samoa, Singapore, Solomon Islands, Taiwan, Taiwan*, Tonga, Tuvalu, Vanuatu, Vietnam'
wpro = [i.strip() for i in wpro.split(',')]
for i in wpro:
    who_region[i] = 'Western Pacific'

In [9]:
# Addding 'WHO Region' & list of countries with missing "WHO Region"

df_wm['WHO Region'] = df_wm['Country/Region'].map(who_region)
df_wm[df_wm['WHO Region'].isna()]['Country/Region'].unique()

array(['French Polynesia', 'French Guiana', 'Guadeloupe', 'Réunion',
       'CAR', 'Mayotte', 'Martinique', 'Aruba', 'Curaçao',
       'Channel Islands', 'Sint Maarten', 'Gibraltar', 'Diamond Princess',
       'Turks and Caicos', 'Saint Martin', 'Faeroe Islands',
       'Isle of Man', 'Cayman Islands', 'Caribbean Netherlands',
       'Brunei ', 'St. Barth', 'St. Vincent Grenadines',
       'British Virgin Islands', 'Macao', 'New Caledonia',
       'Saint Pierre Miquelon', 'Montserrat', 'Falkland Islands'],
      dtype=object)

In [10]:
# Data corrections

for col in df_wm.columns[2:]:
    # replace comma with empty string
    df_wm[col] = df_wm[col].str.replace('[,+ ]', '', regex=True)
    # replace 'N/A' with empty string
    df_wm[col] = df_wm[col].str.replace('N/A', '', regex=False)

# replace empty strings with np.nan
df_wm = df_wm.replace('', np.nan)

df_wm.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
0,USA,North America,331706290,10557037,133427,245777,1322,6595348,41959,3715912,19077,31826,741,160475931,483789,Americas
1,India,Asia,1384900866,8635754,44679,127615,511,8011844,54638,496295,8944,6236,92,119615857,86371,South-EastAsia
2,Brazil,South America,213106404,5701283,25517,162842,204,5064344,NaN,474097,8318,26753,764,21900000,102766,Americas
3,France,Europe,65326081,1829659,22180,42207,857,131920,2185,1655532,4750,28008,646,18249032,279353,Europe
4,Russia,Europe,145957281,1817109,20977,31161,368,1350741,15600,435207,2300,12450,213,65600000,449447,Europe


In [11]:
# Save as "csv" file

df_wm.to_csv('worldometer_data.csv', index=False)